# Code to train an MNIST Classifier

In [1]:
!pip install torchvision

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for torch==2.5.1 from https://files.pythonhosted.org/packages/d1/35/e8b2daf02ce933e4518e6f5682c72fd0ed66c15910ea1fb4168f442b71c4/torch-2.5.1-cp311-cp311-manylinux1_x86_64.whl.metadata
  Using cached torch-2.5.1-cp311-cp311-manylinux1_x86_64.whl.metadata (28 kB)
  Obtaining dependency information for nvidia-cudnn-cu12==9.1.0.70 from https://files.pythonhosted.org/packages/9f/fd/713452cd72343f682b1c7b9321e23829f00b842ceaedcda96e742ea0b0b3/nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Obtaining dependency information for nvidia-nccl-cu12==2.21.5 from https://files.pythonhosted.org/packages/df/99/12cd266d6233f47d00daf3a72739872bdc10267d0383508b0b9c84a18bb6/nvidia_nccl_cu12-2.21.5-py3-none-manylinux2014_x86_64.whl.metadata
  Using cached nvidia_nccl_cu12-2.21.5-

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F

In [3]:
batch_size = 64
learning_rate = 1e-3
num_epochs = 10

In [4]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)) 
])


train_dataset = datasets.MNIST(root='./mnist_data', train=True, transform = transform, download=True)
test_dataset = datasets.MNIST(root='./mnist_data', train=False, transform=transform, download=True)

In [5]:
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=64)
val_loader = DataLoader(test_dataset, batch_size=64)

In [6]:
xtrain, ytrain = next(iter(train_loader))
print(xtrain.shape, ytrain.shape)

torch.Size([64, 1, 28, 28]) torch.Size([64])


# Model

In [7]:
class MNISTClassifier(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(784, 512)
        self.fc2 = nn.Linear(512, 512)
        self.fc3 = nn.Linear(512, 256)
        self.dropout = nn.Dropout(0.1)
        self.fc4 = nn.Linear(256, 128)
        self.fc5 = nn.Linear(128, 10)
        self.relu = nn.ReLU()
    
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc4(x)
        x = self.relu(x)
        x = self.fc5(x)
        x = torch.softmax(x, dim=-1)
        return x

# Training Loop

In [12]:
model = MNISTClassifier()
learning_rate = 1e-3
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = optim.lr_scheduler.StepLR(optimizer, 2, gamma=0.85)

In [13]:
def calculate_num_correct(outputs, labels):
    # outputs.shape: (b,10)
    # labels.shape: (b)
    max_output_vals, max_output_indices = outputs.max(dim=-1)
    correct = torch.where(max_output_indices == labels, 1, 0)
    
    num_correct = torch.sum(correct, dim=-1)
    total = correct.size(0)
    return num_correct, total


def run_training_loop(model, optimizer, train_loader, val_loader, scheduler, num_epochs=10, print_every=200):
    device = next(model.parameters()).device # device that model is stored on
    training_log = {
        'training_loss': [],
        'training_acc': [],
        'validation_loss': [],
        'validation_acc': []
    }
    
    model.train()
    num_iter = 0
    for i in range(num_epochs):
        
        for iter_idx, (xb,yb) in enumerate(train_loader):
            xb, yb = xb.to(device), yb.to(device)
            xb = xb.view(-1, 784)
            out = model(xb) # b, 10
            loss = F.cross_entropy(out, yb)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            num_correct, total = calculate_num_correct(out, yb)
            
            training_log['training_loss'].append((num_iter, loss.item()))
            training_log['training_acc'].append((num_iter, num_correct / total))
            
            if num_iter % print_every == 0:
                print(f'Epoch {i}, iter: {num_iter}: training loss: {training_log["training_loss"][-1][1]}, training acc: {training_log["training_acc"][-1][1]}')
            
            num_iter += 1
        
        
        total_num_correct = 0
        total_val_samples = 0
        total_loss = 0
        num_batches = 0
        
        # run evals
        for iter_idx, (xb, yb) in enumerate(val_loader):
            
            xb, yb = xb.to(device), yb.to(device)
            xb = xb.view(-1, 784)
            out = model(xb)
            loss = F.cross_entropy(out, yb)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            num_iter += 1
            
            num_correct, total = calculate_num_correct(out, yb)
            
            total_num_correct += num_correct
            total_val_samples += total
            total_loss += loss.item()
            
            num_batches += 1

            
            
        training_log['validation_loss'].append((num_iter, loss.item()))
        training_log['validation_acc'].append((num_iter, num_correct / total))
        
        
        scheduler.step()
        
        print(f'Epoch {i}, iter: {num_iter}: validation loss: {training_log["validation_loss"][-1][1]}, validation acc: {training_log["validation_acc"][-1][1]}')

    
    return training_log

In [14]:
run_training_loop(model, optimizer, train_loader, val_loader, scheduler, num_epochs=10)

Epoch 0, iter: 0: training loss: 2.303898811340332, training acc: 0.0625
Epoch 0, iter: 200: training loss: 1.7404704093933105, training acc: 0.71875
Epoch 0, iter: 400: training loss: 1.6941438913345337, training acc: 0.765625
Epoch 0, iter: 600: training loss: 1.7248358726501465, training acc: 0.71875
Epoch 0, iter: 800: training loss: 1.5516202449798584, training acc: 0.90625
Epoch 0, iter: 1095: validation loss: 1.5478267669677734, validation acc: 0.9375
Epoch 1, iter: 1200: training loss: 1.6116087436676025, training acc: 0.859375
Epoch 1, iter: 1400: training loss: 1.5865415334701538, training acc: 0.875
Epoch 1, iter: 1600: training loss: 1.6707532405853271, training acc: 0.796875
Epoch 1, iter: 1800: training loss: 1.5696589946746826, training acc: 0.890625
Epoch 1, iter: 2000: training loss: 1.5519883632659912, training acc: 0.90625
Epoch 1, iter: 2190: validation loss: 1.4611504077911377, validation acc: 1.0
Epoch 2, iter: 2200: training loss: 1.5532480478286743, training acc

{'training_loss': [(0, 2.303898811340332),
  (1, 2.301663398742676),
  (2, 2.300722599029541),
  (3, 2.298607349395752),
  (4, 2.2957191467285156),
  (5, 2.296870231628418),
  (6, 2.2909915447235107),
  (7, 2.28743839263916),
  (8, 2.257431745529175),
  (9, 2.2733426094055176),
  (10, 2.275371789932251),
  (11, 2.188725471496582),
  (12, 2.2323808670043945),
  (13, 2.1879100799560547),
  (14, 2.1379215717315674),
  (15, 2.1807310581207275),
  (16, 2.111475944519043),
  (17, 2.104257345199585),
  (18, 2.1866612434387207),
  (19, 2.045283079147339),
  (20, 2.095831871032715),
  (21, 2.1798534393310547),
  (22, 2.0370898246765137),
  (23, 2.0884451866149902),
  (24, 2.1116597652435303),
  (25, 2.0978055000305176),
  (26, 1.9934695959091187),
  (27, 2.149066209793091),
  (28, 2.087299346923828),
  (29, 2.084160566329956),
  (30, 2.0706589221954346),
  (31, 2.060039520263672),
  (32, 2.18245005607605),
  (33, 2.103854179382324),
  (34, 1.9840469360351562),
  (35, 2.124175786972046),
  (36, 